#### <center> CISB5123 Text Analytics </center>
#### <center> Sem 2 2024/2025 </center>

### <center> Lab Assignment 1 - Web Scraping </center>

### 1) Amirul Farhan bin Kamaruzaman, SW01082374
### 2) Maizatul Aufa binti Zamidi, SW01082394

In [9]:
!pip install selenium

In [10]:
pip show selenium

Name: selenium
Version: 4.29.0
Summary: Official Python bindings for Selenium WebDriver
Home-page: https://www.selenium.dev
Author: 
Author-email: 
License: Apache 2.0
Location: C:\Anaconda\Lib\site-packages
Requires: certifi, trio, trio-websocket, typing_extensions, urllib3, websocket-client
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [47]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Set up WebDriver
chrome_driver_path = "C:/Users/Amirul Farhan/Text Analytics/Lab Assignment 1/chromedriver-win64/chromedriver.exe"
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service)

# Lazada Product URL
url = "https://www.lazada.com.my/products/expose-monitor-27-inch-curved-gaming-monitor-165hz-led-monitor-computer-frameless-murah-1080p-hdmi-wide-screen-display-monitor-i3780653745-s21774949370.html"
driver.get(url)
time.sleep(5)  # Allow time for the page to load

def scroll_to_bottom():
    """Scroll multiple times to load all reviews properly"""
    for _ in range(3):  # Scroll multiple times to ensure loading
        driver.execute_script("window.scrollBy(0, 1000);")
        time.sleep(2)  # Allow time for reviews to load

def extract_reviews():
    """Extract reviews from Lazada"""
    reviews = []
    
    for page in range(1, 6):  # Limit to 5 pages
        print(f"Scraping page {page}...")

        scroll_to_bottom()  # Scroll to load content
        
        try:
            # Explicit wait for review elements
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "item"))
            )

            review_elements = driver.find_elements(By.CLASS_NAME, "item")  # Locate reviews
            
            for review in review_elements:
                try:
                    # Extract Reviewer Name
                    name = review.find_element(By.CLASS_NAME, "middle").find_element(By.TAG_NAME, "span").text.strip()
                    
                    # Extract Review Date
                    date_element = review.find_elements(By.CLASS_NAME, "title")  
                    date = date_element[0].text.strip() if len(date_element) > 0 else "Date not found"
                    
                    # Extract Review Content
                    content_element = review.find_elements(By.CLASS_NAME, "content")
                    content = content_element[0].text.strip() if len(content_element) > 0 else "No review text"
                    
                    # Append extracted data
                    reviews.append([name, date, content])
                except Exception as e:
                    print(f"Skipping review due to error: {e}")
                    continue

            # Find the next page button
            try:
                next_button = driver.find_element(By.CSS_SELECTOR, "button.next-pagination-item.next")
                
                if next_button:
                    # Scroll next button into view
                    driver.execute_script("arguments[0].scrollIntoView();", next_button)
                    time.sleep(1)

                    # Try normal click, if fails use JavaScript
                    try:
                        next_button.click()
                    except:
                        driver.execute_script("arguments[0].click();", next_button)

                    time.sleep(3)  # Allow next page to load

                else:
                    print("'Next' button not found.")
                    break  # Stop if no pagination is found

            except Exception as e:
                print(f"Pagination issue: {e}")
                break  # Stop if no next page available

        except Exception as e:
            print(f"Error: {e}")
            break

    return reviews

# Extract the reviews
data = extract_reviews()

# Save to CSV
if data:
    df = pd.DataFrame(data, columns=["Reviewer Name", "Review Date", "Review Content"])
    df.to_csv("lazada_reviews.csv", index=False, encoding='utf-8')
    print("Reviews successfully saved to lazada_reviews.csv")
else:
    print("No reviews found.")

# Close WebDriver
driver.quit()


Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Reviews successfully saved to lazada_reviews.csv
